In [4]:
import carla
import time
import pandas as pd

# Connect to CARLA
client = carla.Client('localhost', 2000)
client.set_timeout(10.0)
world = client.get_world()
time.sleep(5)  # ⏳ Allow vehicles to spawn

# Get all vehicles
vehicles = world.get_actors().filter('vehicle.*')

# Initialize data storage
collision_data = []
collision_events = {}  # Dictionary to track collisions per vehicle

# Function to handle collision events
def collision_callback(event):
    vehicle_id = event.actor.id
    collision_events[vehicle_id] = True  # Mark vehicle as collided

# Attach collision sensors to vehicles
for vehicle in vehicles:
    blueprint = world.get_blueprint_library().find('sensor.other.collision')
    sensor = world.spawn_actor(blueprint, carla.Transform(), attach_to=vehicle)
    sensor.listen(collision_callback)
    collision_events[vehicle.id] = False  # Initialize as no collision

# Function to check for nearby vehicles
def check_collision_behavior(vehicle, world):
    vehicle_list = world.get_actors().filter('vehicle.*')
    detected_vehicles = 0
    collision_detected = collision_events.get(vehicle.id, False)  # Check collision status

    for other_vehicle in vehicle_list:
        if other_vehicle.id != vehicle.id:
            distance = vehicle.get_location().distance(other_vehicle.get_location())
            if distance < 15:  # Detection threshold
                detected_vehicles += 1

    return detected_vehicles, collision_detected

# Log data for 1 minute
start_time = time.time()
sampling_interval = 3.0  # Log every 3 seconds
last_logged_time = start_time

while time.time() - start_time < 60:  # Run for 1 minute
    world.tick()
    
    if time.time() - last_logged_time >= sampling_interval:
        last_logged_time = time.time()
        
        for vehicle in vehicles:
            detected, collision = check_collision_behavior(vehicle, world)
            collision_data.append({
                "vehicle_id": vehicle.id,
                "time": round(time.time() - start_time, 2),
                "detected_vehicles": detected,
                "collision_detected": 1 if collision else 0
            })

# Save base collision data
df_collision = pd.DataFrame(collision_data)
df_collision.to_csv("collision_data_base.csv", index=False)

print("✅ Base collision data saved!")


✅ Base collision data saved!
